In [1]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [2]:
city = 'SPO'
dias = 30
dias_churn = 7

In [32]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
query = """
SELECT 
    s.identifier_value AS region_code,
    --DATE(fs.order_submitted_date) AS submit_date,
    --cat.parent_description as category,
    --cat.description AS subcategory,
    --dp.card_id,
    --dp.card_description AS product_name,
    dc.source_id AS customer_id,
    SUM(CASE WHEN cat.super_category = 'Fruver' THEN 1 ELSE 0 END) AS all_view, 
    SUM(CASE WHEN cat.super_category = 'Fruver' AND DATE(fs.order_submitted_date) >= DATE(current_date - {dias_churn}) THEN 1 ELSE 0 END) AS last_view,
    --all_view*last_view AS check,
    CASE WHEN all_view > 0 AND last_view = 0 THEN 1 ELSE 0 END AS decision,
    COUNT( DISTINCT fs.order_id) AS orders,
    SUM(fs.product_quantity_x_step_unit) AS cant,
    SUM(fs.gmv_pxq) AS gmv_usd,
    

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND s.identifier_value = '{ciudad}'
    AND DATE(fs.order_submitted_date) > current_date - {dias}
GROUP BY 1,2--,3--,4,5,6,7
""".format(ciudad=city,dias=dias,dias_churn=dias_churn)

df = run_read_dwd_query(query)

In [9]:
query = """
SELECT 
    s.identifier_value AS region_code,
    --DATE(fs.order_submitted_date) AS submit_date,
    cat.parent_description as category,
    cat.description AS subcategory,
    dp.card_id,
    dp.card_description AS product_name,
    COUNT(DISTINCT customer_id) AS customers
    --,LISTAGG(DISTINCT dc.source_id, ',') AS customer_list
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND s.identifier_value = '{ciudad}'
    AND DATE(fs.order_submitted_date) > current_date - {dias}
    AND dc.source_id IN {customers}
    AND cat.super_category = 'Fruver'
GROUP BY 1,2,3,4,5--,6,7
ORDER BY 6 DESC
""".format(ciudad=city,dias=dias,customers= tuple(df.loc[df.decision == 1].customer_id.unique()))

df2 = run_read_dwd_query(query)

In [58]:
def concatenate_skus(card_id):
    try:
        sku_value = '#'.join(frida_products.loc[frida_products['id_tarjeta'] == card_id, 'skus'].values)
    except IndexError:
        sku_value = ''
    return sku_value

In [19]:
from sklearn.model_selection import train_test_split

def create_groups(offer):
    customers = offer.customer_id.unique()
    print("Total length: ", len(customers))
    
    # Split into control, test1, and test2 (1/3 each)
    customer_control, customer_test = train_test_split(customers, test_size=0.5)

    alpha = 0.0001
    aux = 0
    
    while abs(1 - (offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean() / offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:        
        # Re-split if the condition is not met
        customer_control, customer_test = train_test_split(customers, test_size=0.5)
        
        aux += 1
        if aux == 10000:
            print(abs(1 - (offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean() / offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())))
            alpha += 0.01
            aux = 0
    
    print(abs(1 - (offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean() / offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())))
    
    return customer_control.tolist(), customer_test.tolist()

In [20]:
control,test = create_groups(df.loc[df.decision == 1])

Total length:  4266
1.2372302957475156e-05


In [30]:
global_segment_list = [] #//for procesing

d = {'name':"EXP_CHURN_FRUVER_"+city+"_"+"_grupo_"+'_test', 'customersIds': list(test),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(70))).strftime('%Y-%m-%d')}
f = {'name':"EXP_CHURN_FRUVER_"+city+"_"+"_grupo_"+"_control", 'customersIds': list(control),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(70))).strftime('%Y-%m-%d')}

global_segment_list.append(d)
global_segment_list.append(f) 

In [33]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [34]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [86]:
sgt

[(623402, 'EXP_CHURN_FRUVER_SPO__grupo_test'),
 (623403, 'EXP_CHURN_FRUVER_SPO__grupo__control')]

In [27]:
df2 = df2.iloc[:20,:].reset_index(drop=True)

In [28]:
df2["offer_name"] = "EXP_FRUVER_"+city+"_"+df2.product_name.replace(" ","_").replace(",",".")+"_20%_FEB24"

In [83]:
csv_file = {} 
csv_file['offer_name'] = df2.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = 20
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(7))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 0
csv_file['max_uses_per_customer'] = 0
csv_file['customer_segment_id'] = sgt[0][0] #[None]
csv_file['sku_id'] = df2['card_id'].apply(concatenate_skus)
csv_file['segment_behaviour'] = 'INCLUDED'
csv_file['offer_type'] = 'OTHERS'
csv_file['supplier_discount'] = 0

global_offer_csv = pd.DataFrame(csv_file)


In [85]:
global_offer_csv.to_csv("discount.csv")